This is an example of SimSwap on processing video with multiple faces with designated sources.

Code path: https://github.com/neuralchen/SimSwap
Paper path: https://arxiv.org/pdf/2106.06340v1.pdf.

In [1]:
## make sure you are using a runtime with GPU
## you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

Thu Jul 31 15:56:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

All file changes make by this notebook are temporary.
You can try to mount your own google drive to store files if you wang.


In [2]:
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

fatal: destination path 'SimSwap' already exists and is not an empty directory.
Already up to date.


In [3]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

  Using cached imageio-2.4.1-py3-none-any.whl
  Attempting uninstall: imageio
    Found existing installation: imageio 2.37.0
    Uninstalling imageio-2.37.0:
      Successfully uninstalled imageio-2.37.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires imageio!=2.35.0,>=2.33, but you have imageio 2.4.1 which is incompatible.
moviepy 1.0.3 requires imageio<3.0,>=2.5; python_version >= "3.4", but you have imageio 2.4.1 which is incompatible.


In [4]:
import os
os.chdir("SimSwap")
!ls

 antelope.zip	       predict.py
 arcface_model	       README.md
 checkpoints	       SimSwap
 checkpoints.zip      'SimSwap colab.ipynb'
 cog.yaml	       simswaplogo
 crop_224	       test_one_image.py
 data		       test_video_swapmulti.py
 demo_file	       test_video_swap_multispecific.py
 docs		       test_video_swapsingle.py
 download-weights.sh   test_video_swapspecific.py
 insightface_func      test_wholeimage_swapmulti.py
 LICENSE	       test_wholeimage_swap_multispecific.py
 models		       test_wholeimage_swapsingle.py
 MultiSpecific.ipynb   test_wholeimage_swapspecific.py
 options	       train.ipynb
 output		       train.py
 parsing_model	       util
 pg_modules


In [5]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

In [6]:
# import gdown;

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# gdown.download('https://drive.google.com/file/d/1zUUlvIQvLlho6nymx6TwNFjy4OTTKt3d', output='./arcface_model/arcface_checkpoint.tar')
# gdown.download('https://drive.google.com/file/d/1QZ8U7c9rFKTIVdAKGu2hueeGGSGIIqli', output='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

--2025-07-31 15:57:08--  https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/374891081/f01468b3-446b-4867-8c78-6d496183f9e6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-31T16%3A54%3A48Z&rscd=attachment%3B+filename%3Darcface_checkpoint.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-31T15%3A54%3A02Z&ske=2025-07-31T16%3A54%3A48Z&sks=b&skv=2018-11-09&sig=WT7GUrNWsVq7nikZanhaSAOVf8FOrDqGd6U3JtEokWQ%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1Mzk3NzcyOCwibmJmIjoxNzUzOTc3NDI4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVj

In [7]:
# !wget --no-check-certificate "https://drive.google.com/file/d/1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln/view?usp=sharing" -O antelope.zip
# !unzip ./antelope.zip -d ./insightface_func/models/

!pip install gdown
!gdown 1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln -O antelope.zip
!mkdir -p insightface_func/models/antelope
!unzip -q antelope.zip -d insightface_func/models/antelope/


Downloading...
From (original): https://drive.google.com/uc?id=1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln
From (redirected): https://drive.google.com/uc?id=1tNRVUUoYiJraBDpoCoHIZbWaOZ1TF1Ln&confirm=t&uuid=a5990af6-a5ce-4a2d-88de-de644f59b555
To: /content/SimSwap/antelope.zip
100% 248M/248M [00:05<00:00, 42.4MB/s]
replace insightface_func/models/antelope/antelope/glintr100.onnx? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [8]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap_multispecific import video_swap
import os
import glob

In [9]:
def lcm(a, b): return abs(a * b) / fractions.gcd(a, b) if a and b else 0

transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [10]:
!ls ./checkpoints

people


In [11]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.multisepcific_dir = './demo_file/multispecific' ## or replace it with folder from your own google drive
                           ## and remember to follow the dir structure in usage.md
opt.video_path = './demo_file/multi_people_1080p.mp4' ## or replace it with video from your own google drive
opt.output_path = './output/multi_test_multispecific.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.name = 'people'
opt.isTrain = False
opt.use_mask = True  ## new feature up-to-date

crop_size = opt.crop_size


------------ Options -------------
Arc_path: arcface_model/arcface_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
crop_size: 512
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
f: /root/.local/share/jupyter/runtime/kernel-1e67c10d-d8bc-4e8d-af50-840b31c1a623.json
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
id_thres: 0.03
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
multisepcific_dir: ./demo_file/multispecific
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
no_simswaplogo: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: 

In [13]:
pic_specific = opt.pic_specific_path
crop_size = opt.crop_size
multisepcific_dir = opt.multisepcific_dir

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))
# The specific person to be swapped(source)
source_specific_id_nonorm_list = []
source_path = os.path.join(multisepcific_dir,'SRC_*')
source_specific_images_path = sorted(glob.glob(source_path))

with torch.no_grad():
    for source_specific_image_path in source_specific_images_path:
        specific_person_whole = cv2.imread(source_specific_image_path)
        specific_person_align_crop, _ = app.get(specific_person_whole,crop_size)
        specific_person_align_crop_pil = Image.fromarray(cv2.cvtColor(specific_person_align_crop[0],cv2.COLOR_BGR2RGB))
        specific_person = transformer_Arcface(specific_person_align_crop_pil)
        specific_person = specific_person.view(-1, specific_person.shape[0], specific_person.shape[1], specific_person.shape    [2])
        # convert numpy to tensor
        specific_person = specific_person.cuda()
        #create latent id
        specific_person_downsample = F.interpolate(specific_person, size=(112,112))
        specific_person_id_nonorm = model.netArc(specific_person_downsample)
        source_specific_id_nonorm_list.append(specific_person_id_nonorm.clone())

    # The person who provides id information (list)
    target_id_norm_list = []
    target_path = os.path.join(multisepcific_dir,'DST_*')
    target_images_path = sorted(glob.glob(target_path))

    for target_image_path in target_images_path:
        img_a_whole = cv2.imread(target_image_path)
        img_a_align_crop, _ = app.get(img_a_whole,crop_size)
        img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB))
        img_a = transformer_Arcface(img_a_align_crop_pil)
        img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
        # convert numpy to tensor
        img_id = img_id.cuda()
        #create latent id
        img_id_downsample = F.interpolate(img_id, size=(112,112))
        latend_id = model.netArc(img_id_downsample)
        latend_id = F.normalize(latend_id, p=2, dim=1)
        target_id_norm_list.append(latend_id.clone())

    assert len(target_id_norm_list) == len(source_specific_id_nonorm_list), "The number of images in source and target  directory must be same !!!"
    video_swap(opt.video_path, target_id_norm_list,source_specific_id_nonorm_list, opt.id_thres, \
        model, app, opt.output_path,temp_results_dir=opt.temp_path,no_simswaplogo=opt.no_simswaplogo,use_mask=opt.use_mask)

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL models.arcface_models.ResNet was not an allowed global by default. Please use `torch.serialization.add_safe_globals([ResNet])` or the `torch.serialization.safe_globals([ResNet])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weigh